In [89]:
import numpy as np
import pandas as pd
from pandas import read_csv, set_option

from sklearn.model_selection import cross_val_score, train_test_split, RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.utils import resample, shuffle

#### 혹시 모르니 써볼까 싶은건 일단 다 불러봅니다

In [90]:
import warnings
warnings.filterwarnings('ignore')

#### 경고는 자신감 하락의 원천이라 무시해줍니다

In [91]:
X_train_df = read_csv('train.csv')
X_test_df = read_csv('test.csv')

In [92]:
X_train_df.describe()

,id,Overall Qual,Gr Liv Area,Garage Cars,Garage Area,Total Bsmt SF,1st Flr SF,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
count,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000
mean,675.500000,6.208889,1513.542222,1.870370,502.014815,1082.644444,1167.474074,1.560741,1972.987407,1985.099259,1978.471852,186406.312593
std,389.855743,1.338015,487.523239,0.652483,191.389956,384.067713,375.061407,0.551646,29.307257,20.153244,25.377278,78435.424758
min,1.000000,2.000000,480.000000,1.000000,100.000000,105.000000,480.000000,0.000000,1880.000000,1950.000000,1900.000000,12789.000000
25%,338.250000,5.000000,1144.000000,1.000000,368.000000,816.000000,886.250000,1.000000,1955.000000,1968.000000,1961.000000,135000.000000
50%,675.500000,6.000000,1445.500000,2.000000,484.000000,1009.000000,1092.500000,2.000000,1976.000000,1993.000000,1978.500000,165375.000000
75%,1012.750000,7.000000,1774.500000,2.000000,588.000000,1309.500000,1396.500000,2.000000,2002.000000,2004.000000,2002.000000,217875.000000
max,1350.000000,10.000000,4476.000000,5.000000,1488.000000,2660.000000,2898.000000,4.000000,2010.000000,2010.000000,2207.000000,745000.000000


In [93]:
X_train_df[X_train_df['Garage Yr Blt']>=2022]

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
254,255,8,1564,Gd,2,502,Ex,1546,1564,Gd,2,2006,2007,2207,267300


In [94]:
X_train_df[X_train_df['Year Built']==2006]

,id,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
1,2,7,1352,Gd,2,466,Gd,1352,1352,Ex,2,2006,2007,2006,194000
19,20,6,1326,Gd,2,427,Gd,1326,1326,Gd,2,2006,2006,2006,174190
67,68,8,1460,Gd,2,480,Gd,1460,1460,Ex,2,2006,2007,2006,247000
97,98,6,1495,Gd,2,440,Gd,738,738,Gd,2,2006,2006,2006,171925
102,103,8,1324,Gd,2,550,Gd,1324,1324,Gd,2,2006,2006,2006,254000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1288,1289,7,1977,Gd,3,574,Gd,982,982,Gd,2,2006,2006,2006,250000
1327,1328,8,1634,Gd,3,866,Gd,1626,1634,Gd,2,2006,2006,2006,250000
1328,1329,8,1370,Gd,2,484,Gd,1370,1370,Gd,2,2006,2006,2006,205000
1336,1337,10,2698,Ex,3,736,Ex,1850,1850,Ex,2,2006,2006,2006,535000


#### Describe() 확인 결과, Grage Yr Blt 값 중 하나가 비정상적으로 큽니다
#### 2007을 2207로 잘못 쓴게 아닌가 직관적으로 판단해봅니다
#### 그런데 Year Built 연도가 2006인 경우, 보통 차고를 2006년에 같이 짓는 경우가 데이터 상 대부분입니다
#### 2006으로 바꿔봤더니 성능이 조금 더 좋아져서 2006으로 변경했습니다 (사실 별 차이 없습니다)

In [95]:
X_train_df.loc[(X_train_df['Garage Yr Blt'] == 2207), 'Garage Yr Blt']=2006

In [96]:
grade = ['Ex', 'Gd', 'TA', 'Fa', 'Po']
j=5
for i in grade:
    X_train_df.loc[(X_train_df['Exter Qual'] == i), 'Exter Qual'] = j
    X_train_df.loc[(X_train_df['Kitchen Qual'] == i), 'Kitchen Qual'] = j
    X_train_df.loc[(X_train_df['Bsmt Qual'] == i), 'Bsmt Qual'] = j
    X_test_df.loc[(X_test_df['Exter Qual'] == i), 'Exter Qual'] = j
    X_test_df.loc[(X_test_df['Kitchen Qual'] == i), 'Kitchen Qual'] = j
    X_test_df.loc[(X_test_df['Bsmt Qual'] == i), 'Bsmt Qual'] = j
    j-=1

#### 범주형 변수를 수치형으로 바꿔줍니다
#### 순간 라벨 인코딩이 생각나지 않아(핑곕니다)
#### 조악한 코딩실력으로 반복문을 구사해봅니다

In [97]:
X_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1350 non-null   int64 
 1   Overall Qual    1350 non-null   int64 
 2   Gr Liv Area     1350 non-null   int64 
 3   Exter Qual      1350 non-null   object
 4   Garage Cars     1350 non-null   int64 
 5   Garage Area     1350 non-null   int64 
 6   Kitchen Qual    1350 non-null   object
 7   Total Bsmt SF   1350 non-null   int64 
 8   1st Flr SF      1350 non-null   int64 
 9   Bsmt Qual       1350 non-null   object
 10  Full Bath       1350 non-null   int64 
 11  Year Built      1350 non-null   int64 
 12  Year Remod/Add  1350 non-null   int64 
 13  Garage Yr Blt   1350 non-null   int64 
 14  target          1350 non-null   int64 
dtypes: int64(12), object(3)
memory usage: 158.3+ KB


#### 어딘가 문제가 있음을 직감합니다

In [98]:
X_train_df['Exter Qual'] = pd.to_numeric(X_train_df['Exter Qual'])
X_train_df['Kitchen Qual'] = pd.to_numeric(X_train_df['Kitchen Qual'])
X_train_df['Bsmt Qual'] = pd.to_numeric(X_train_df['Bsmt Qual'])
X_test_df['Exter Qual'] = pd.to_numeric(X_test_df['Exter Qual'])
X_test_df['Kitchen Qual'] = pd.to_numeric(X_test_df['Kitchen Qual'])
X_test_df['Bsmt Qual'] = pd.to_numeric(X_test_df['Bsmt Qual'])

#### 결국 numeric함수까지 쓰면서 두 번에 걸쳐 수치형 변환을 마칩니다

In [99]:
X = X_train_df.drop(['id', 'target'], axis=1)
y = X_train_df.target
X_test = X_test_df.drop(['id'], axis=1)

In [100]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Overall Qual    1350 non-null   int64
 1   Gr Liv Area     1350 non-null   int64
 2   Exter Qual      1350 non-null   int64
 3   Garage Cars     1350 non-null   int64
 4   Garage Area     1350 non-null   int64
 5   Kitchen Qual    1350 non-null   int64
 6   Total Bsmt SF   1350 non-null   int64
 7   1st Flr SF      1350 non-null   int64
 8   Bsmt Qual       1350 non-null   int64
 9   Full Bath       1350 non-null   int64
 10  Year Built      1350 non-null   int64
 11  Year Remod/Add  1350 non-null   int64
 12  Garage Yr Blt   1350 non-null   int64
dtypes: int64(13)
memory usage: 137.2 KB


In [101]:
def feature(data):
  df = data.copy()
  df['Year Gap Remod'] = df['Year Remod/Add'] - df['Year Built']
  df['Car Area'] = df['Garage Area']/df['Garage Cars']
  df['2nd flr SF'] = df['Gr Liv Area'] - df['1st Flr SF']
  df['2nd flr'] = df['2nd flr SF'].apply(lambda x : 1 if x > 0 else 0)
  df['Total SF'] = df[['Gr Liv Area',"Garage Area", "Total Bsmt SF"]].sum(axis=1)
  df['Sum Qual'] = df[["Exter Qual", "Kitchen Qual", "Overall Qual"]].sum(axis=1)
  df['Garage InOut'] = df.apply(lambda x : 1 if x['Gr Liv Area'] != x['1st Flr SF'] else 0, axis=1)
  return df

#### [동화책] 님의 파생변수를 활용합니다 (정말 감사합니다)
#### 파생변수들 간의 교호작용도나 다중공선성의 우려로 수동후진제거법을 통해 최적의 변수 조합을 찾습니다 (수동후진제거법 : 노가다법)
#### → 결론 : 다 쓰는게 제일 좋습니다

In [102]:
X = feature(X)
X_test0 = feature(X_test)

In [103]:
X_train, X_valid, y_train, y_valid=train_test_split(X, y, test_size=0.2, random_state=0)

#### 학습셋과 검증셋을 나누어 줍니다

In [104]:
from sklearn.metrics import make_scorer

def NMAE(true, pred):
    mae = np.mean(np.abs(true - pred))
    score = mae / np.mean(np.abs(true))
    return score

nmae_score = make_scorer(NMAE, greater_is_better=False)

In [105]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import AdaBoostRegressor
from catboost import CatBoostRegressor, Pool
from ngboost import NGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet

#### 알고 있는 회귀모델은 모두 사용해봅니다 (사실 몰랐던 것도 있습니다_코드 공유해주신 모든 분들 감사합니다)

In [106]:
etr=ExtraTreesRegressor(n_estimators=200)
lr=LinearRegression()
rg=Ridge()
rf=RandomForestRegressor(random_state = 42, criterion = 'mae')
gbr=GradientBoostingRegressor(random_state = 42, max_depth = 4, learning_rate=0.09, loss='huber', n_estimators=130)
xgb=XGBRegressor(random_state = 42, learning_rate=0.05, n_estimators=200)
lgb=LGBMRegressor(random_state = 42, learning_rate=0.05, n_estimators=200)
ada=AdaBoostRegressor(random_state = 42, learning_rate=0.03, n_estimators=100)
hgb=HistGradientBoostingRegressor(random_state=42, scoring=nmae_score, learning_rate=0.09)
ngb=NGBRegressor(random_state = 42, n_estimators = 500, verbose = 0, learning_rate = 0.03)
cat_reg=CatBoostRegressor(depth = 4, random_state = 42, loss_function = 'MAE', n_estimators = 3000, learning_rate = 0.03, verbose = 0)

#### 하이퍼파라미터를 수정해보며 최적의 조건을 찾아봅니다
#### (개발자처럼 보이기위해 random_state는 꼭 42를 넣어줍니다)

In [107]:
etr.fit(X_train,y_train)
lr.fit(X_train,y_train)
rg.fit(X_train,y_train)
rf.fit(X_train,y_train)
gbr.fit(X_train,y_train)
xgb.fit(X_train,y_train)
lgb.fit(X_train,y_train)
ada.fit(X_train,y_train)
hgb.fit(X_train,y_train)
ngb.fit(X_train,y_train)
cat_reg.fit(X_train,y_train)

In [108]:
y_etr=etr.predict(X_valid)
y_lr=lr.predict(X_valid)
y_rg=rg.predict(X_valid)
y_rf=rf.predict(X_valid)
y_gbr=gbr.predict(X_valid)
y_xgb=xgb.predict(X_valid)
y_lgb=lgb.predict(X_valid)
y_ada=ada.predict(X_valid)
y_hgb=hgb.predict(X_valid)
y_ngb=ngb.predict(X_valid)
y_cat=cat_reg.predict(X_valid)

In [109]:
score_etr=NMAE(y_valid, y_etr)
score_lr=NMAE(y_valid, y_lr)
score_rg=NMAE(y_valid, y_rg)
score_rf=NMAE(y_valid, y_rf)
score_gbr=NMAE(y_valid, y_gbr)
score_xgb=NMAE(y_valid, y_xgb)
score_lgb=NMAE(y_valid, y_lgb)
score_ada=NMAE(y_valid, y_ada)
score_hgb=NMAE(y_valid, y_hgb)
score_ngb=NMAE(y_valid, y_ngb)
score_cat=NMAE(y_valid, y_cat)

In [110]:
print(score_etr)
print(score_lr)
print(score_rg)
print(score_rf)
print(score_gbr)
print(score_xgb)
print(score_lgb)
print(score_ada)
print(score_hgb)
print(score_ngb)
print(score_cat)

0.09710679251995159
0.11708781373271379
0.1170358457879726
0.09570322997436387
0.09304870273394181
0.09530635456146015
0.09943048864861777
0.1148831783103004
0.09960593628193293
0.09830163250820703
0.09357727386707837


#### 나름 좋은 결과를 보이고 있는 모델들만 선정해줍니다 : ETR+RF+GBR+XGB+LGB+HGB+NGB+CAT

In [111]:
from sklearn.model_selection import KFold
kf = KFold(n_splits = 10, random_state = 42, shuffle = True)

#### Kfold로 fold별 val값, predict값을 평균내어 일반화 성능을 끌어올립니다

In [112]:
X = X_train_df.drop(['id', 'target'], axis=1)
y = np.log1p(X_train_df.target)
X_test = X_test_df.drop(['id'], axis=1)

In [113]:
def feature(data):
  df = data.copy()
  df['Year Gap Remod'] = df['Year Remod/Add'] - df['Year Built']
  df['Car Area'] = df['Garage Area']/df['Garage Cars']
  df['2nd flr SF'] = df['Gr Liv Area'] - df['1st Flr SF']
  df['2nd flr'] = df['2nd flr SF'].apply(lambda x : 1 if x > 0 else 0)
  df['Total SF'] = df[['Gr Liv Area',"Garage Area", "Total Bsmt SF"]].sum(axis=1)
  df['Sum Qual'] = df[["Exter Qual", "Kitchen Qual", "Overall Qual"]].sum(axis=1)
  df['Garage InOut'] = df.apply(lambda x : 1 if x['Gr Liv Area'] != x['1st Flr SF'] else 0, axis=1)
  return df

In [114]:
X = feature(X)
target = feature(X_test)

#### [기세현] 님의 코드를 참고하여 fold별 평균내는 방식을 활용합니다 (감사합니다!)

In [115]:
# ExtraTreesRegression
etr_pred = np.zeros(target.shape[0])
etr_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    etr = ExtraTreesRegressor(n_estimators=200)
    etr.fit(tr_x, tr_y)
    
    val_pred = np.expm1(etr.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    etr_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    target_data = Pool(data = target, label = None)
    fold_pred = etr.predict(target) / 10
    etr_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(etr_val)}')

1 FOLD NMAE = 0.08348082344654736
2 FOLD NMAE = 0.09926120262646408
3 FOLD NMAE = 0.0985187956273541
4 FOLD NMAE = 0.11592711095418551
5 FOLD NMAE = 0.08859707200721012
6 FOLD NMAE = 0.0917048363253914
7 FOLD NMAE = 0.088818592045209
8 FOLD NMAE = 0.09266628380580993
9 FOLD NMAE = 0.09633600419116427
10 FOLD NMAE = 0.10570491401069537
10FOLD Mean of NMAE = 0.09610156350400312


In [116]:
# RandomForestRegressor
rf_pred = np.zeros(target.shape[0])
rf_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    rf = RandomForestRegressor(random_state = 42, criterion = 'mae')
    rf.fit(tr_x, tr_y)
    
    val_pred = np.expm1(rf.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    rf_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    fold_pred = rf.predict(target) / 10
    rf_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(rf_val)}')

1 FOLD NMAE = 0.08585678827973406
2 FOLD NMAE = 0.09632028410805153
3 FOLD NMAE = 0.09554507250507027
4 FOLD NMAE = 0.11977174230217227
5 FOLD NMAE = 0.09064243907754439
6 FOLD NMAE = 0.09381156996589246
7 FOLD NMAE = 0.09032593532044049
8 FOLD NMAE = 0.09432661903251707
9 FOLD NMAE = 0.09595347553328439
10 FOLD NMAE = 0.10419357109533382
10FOLD Mean of NMAE = 0.09667474972200407


In [117]:
# GradientBoostingRegressor
gbr_pred = np.zeros(target.shape[0])
gbr_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    gbr = GradientBoostingRegressor(random_state = 42, max_depth = 4, learning_rate=0.09, loss='huber', n_estimators=130)
    gbr.fit(tr_x, tr_y)
    
    val_pred = np.expm1(gbr.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    gbr_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    fold_pred = gbr.predict(target) / 10
    gbr_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(gbr_val)}')

1 FOLD NMAE = 0.08083175694870585
2 FOLD NMAE = 0.10135722921587453
3 FOLD NMAE = 0.09723700154112937
4 FOLD NMAE = 0.11562379932443151
5 FOLD NMAE = 0.08635983383473296
6 FOLD NMAE = 0.0929372744667634
7 FOLD NMAE = 0.09779780943576759
8 FOLD NMAE = 0.09119687509116622
9 FOLD NMAE = 0.09222487890062216
10 FOLD NMAE = 0.10175773224288907
10FOLD Mean of NMAE = 0.09573241910020827


In [118]:
# xgbr
xgb_pred = np.zeros(target.shape[0])
xgb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    xgb = XGBRegressor(random_state = 42, learning_rate=0.05, n_estimators=200)
    xgb.fit(tr_x, tr_y)
    
    val_pred = np.expm1(xgb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    xgb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    fold_pred = xgb.predict(target) / 10
    xgb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(xgb_val)}')

1 FOLD NMAE = 0.08497414714282708
2 FOLD NMAE = 0.09932995637104627
3 FOLD NMAE = 0.09879971207971086
4 FOLD NMAE = 0.11210960196424802
5 FOLD NMAE = 0.08818492276057927
6 FOLD NMAE = 0.09549798430845158
7 FOLD NMAE = 0.09492628770739124
8 FOLD NMAE = 0.0914800628396734
9 FOLD NMAE = 0.09204532801478633
10 FOLD NMAE = 0.1038036676156383
10FOLD Mean of NMAE = 0.09611516708043524


In [119]:
# lgbr
lgb_pred = np.zeros(target.shape[0])
lgb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    lgb = LGBMRegressor(random_state = 42, learning_rate=0.05, n_estimators=200)
    lgb.fit(tr_x, tr_y)
    
    val_pred = np.expm1(lgb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    lgb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    fold_pred = lgb.predict(target) / 10
    lgb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(lgb_val)}')

1 FOLD NMAE = 0.08145414725180379
2 FOLD NMAE = 0.09553405093358855
3 FOLD NMAE = 0.09590850306577946
4 FOLD NMAE = 0.11384534486886567
5 FOLD NMAE = 0.08985131719867953
6 FOLD NMAE = 0.09724316254882125
7 FOLD NMAE = 0.0886541999982499
8 FOLD NMAE = 0.09740754210583938
9 FOLD NMAE = 0.0902262335304943
10 FOLD NMAE = 0.10531098676479249
10FOLD Mean of NMAE = 0.09554354882669143


In [120]:
# HGBRegressor
hgbr_pred = np.zeros(target.shape[0])
hgbr_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    hgbr = HistGradientBoostingRegressor(random_state=42, scoring=nmae_score, learning_rate=0.09)
    hgbr.fit(tr_x, tr_y)
    
    val_pred = np.expm1(hgbr.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    hgbr_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    fold_pred = hgbr.predict(target) / 10
    hgbr_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(hgbr_val)}')

1 FOLD NMAE = 0.08251162748662036
2 FOLD NMAE = 0.09670328735783504
3 FOLD NMAE = 0.09760392562366041
4 FOLD NMAE = 0.11314114266001851
5 FOLD NMAE = 0.08769244264835112
6 FOLD NMAE = 0.0955066341399757
7 FOLD NMAE = 0.08873382593420542
8 FOLD NMAE = 0.09812719865964409
9 FOLD NMAE = 0.09158364647495551
10 FOLD NMAE = 0.10595075636250212
10FOLD Mean of NMAE = 0.09575544873477684


In [121]:
# NGBRegressor
ngb_pred = np.zeros(target.shape[0])
ngb_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    ngb = NGBRegressor(random_state = 42, n_estimators = 500, verbose = 0, learning_rate = 0.03)
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 300)
    
    val_pred = np.expm1(ngb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    ngb_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    target_data = Pool(data = target, label = None)
    fold_pred = ngb.predict(target) / 10
    ngb_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(ngb_val)}')

1 FOLD NMAE = 0.07562783131946062
2 FOLD NMAE = 0.10206763392271655
3 FOLD NMAE = 0.09823393376837301
4 FOLD NMAE = 0.11048677626410351
5 FOLD NMAE = 0.08741377103635795
6 FOLD NMAE = 0.09882582524819147
7 FOLD NMAE = 0.09614527205418492
8 FOLD NMAE = 0.09081635763194823
9 FOLD NMAE = 0.08957751048511726
10 FOLD NMAE = 0.10353727761567867
10FOLD Mean of NMAE = 0.09527321893461323


In [122]:
# cat
cat_pred = np.zeros(target.shape[0])
cat_val = []
for n, (tr_idx, val_idx) in enumerate(kf.split(X, y)) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], np.expm1(y.iloc[val_idx])
    
    tr_data = Pool(data = tr_x, label = tr_y)
    val_data = Pool(data = val_x, label = val_y)
    
    cat = CatBoostRegressor(depth = 4, random_state = 42, loss_function = 'MAE', n_estimators = 3000, learning_rate = 0.03, verbose = 0)
    cat.fit(tr_data, eval_set = val_data, early_stopping_rounds = 750, verbose = 1000)
    
    val_pred = np.expm1(cat.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    cat_val.append(val_nmae)
    print(f'{n + 1} FOLD NMAE = {val_nmae}')
    
    target_data = Pool(data = target, label = None)
    fold_pred = cat.predict(target) / 10
    cat_pred += fold_pred
print(f'10FOLD Mean of NMAE = {np.mean(cat_val)}')

0:	learn: 0.2938486	test: 184599.8961000	best: 184599.8961000 (0)	total: 2.43ms	remaining: 7.28s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 184599.8409
bestIteration = 145

Shrink model to first 146 iterations.
1 FOLD NMAE = 0.08095120214950641
0:	learn: 0.2941968	test: 182774.3370022	best: 182774.3370022 (0)	total: 14.8ms	remaining: 44.5s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 182774.3178
bestIteration = 88

Shrink model to first 89 iterations.
2 FOLD NMAE = 0.11446639489896013
0:	learn: 0.2888881	test: 196831.3033814	best: 196831.3033814 (0)	total: 1.62ms	remaining: 4.87s
1000:	learn: 0.0668221	test: 196831.2192248	best: 196831.2179529 (831)	total: 1.38s	remaining: 2.75s
Stopped by overfitting detector  (750 iterations wait)

bestTest = 196831.218
bestIteration = 831

Shrink model to first 832 iterations.
3 FOLD NMAE = 0.09267928462268607
0:	learn: 0.2909768	test: 172746.7622039	best: 172746.7622039 (0)	total: 8.8ms	remaining: 26.4s

In [123]:
# 검증 성능 확인하기
val_list = [etr_val, rf_val, gbr_val, xgb_val, lgb_val, hgbr_val, ngb_val, cat_val]
for val in val_list :
  print("{:.8f}".format(np.mean(val))) 

0.09610156
0.09667475
0.09573242
0.09611517
0.09554355
0.09575545
0.09527322
0.12496325


#### 확인 결과, 나머지 성능은 모두 괜찮으나, catboost 성능이 떨어집니다 → 그래도 이대로 보내긴 아쉽습니다

In [124]:
# 제출용 sample 파일을 불러옵니다.
submission = pd.read_csv('sample_submission.csv')
submission.head()

,id,target
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [125]:
predict_cat=cat_reg.predict(X_test0)

#### kfold 하기 전 catboost 모델이라도 불러와서 써줍니다

In [126]:
submission['target'] = (np.expm1(etr_pred) + np.expm1(rf_pred) + np.expm1(gbr_pred) + np.expm1(xgb_pred) + np.expm1(hgbr_pred) + np.expm1(ngb_pred) + predict_cat) / 7
submission

,id,target
0,1,334599.210352
1,2,128235.885117
2,3,176439.511139
3,4,258568.438964
4,5,133662.359152
...,...,...
1345,1346,337677.930450
1346,1347,124889.767898
1347,1348,83669.377571
1348,1349,188830.044902


#### exp으로 cat값과 상태를 맞춰주고서 산술평균을 취해줍니다 (여러번 시도중 LGB는 제출결과 성능이 더 낮아져 제외)

In [127]:
submission.to_csv("submission.csv", index=False)

#### 제출. 끝.